# Data Science Project: Planning Report
### Group Project 003 32 Jeffrey Deng

### Introduction
This project analyzes data collected from a Minecraft research server run by a UBC Computer Science group (Frank Wood). The dataset comprises two linked tables: players.csv (one row per player with profile and demographics) and sessions.csv (one row per play session with timestamps). These logs were gathered to support real operational needs—targeting recruitment and planning server capacity.

### Broad Question
What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

### Specific Question
Can a player’s experience level, total played hours, age, and gender predict whether they subscribe to the newsletter?